# タスク 2: テキスト生成を実行する

このノートブックでは、サポートエンジニアから受けたカスタマーサービスの品質について否定的なフィードバックを寄せた顧客に、大規模言語モデル (LLM) を使用してメール返信を生成する方法を学習します。このノートブックでは、顧客の以前のメールに基づいて、お礼の言葉を記載したメールを生成します。Amazon Nova Lite モデルを使用するには、Boto3 クライアントで Amazon Bedrock API を使用します。

このタスクで使用されるプロンプトは、ゼロショットプロンプトと呼ばれます。ゼロショットプロンプトでは、言語モデルに対し、タスクまたは必要な出力を平易な言葉で記述します。次に、モデルは事前にトレーニングされた知識と機能を使用して、提供されたプロンプトのみに基づいて応答を生成するか、タスクを完了します。

#### シナリオ
あなたは AnyCompany のカスタマーサービスマネージャー、Bob です。一部の顧客はカスタマーサービスに満足しておらず、カスタマーサポートエンジニアが提供するサービスについて否定的なフィードバックを返しています。さて、そのような顧客には、サービスの質が悪かったことをおわびし、信頼を取り戻すような対応をしたいものです。大量のメールを生成するには、LLM の助けが必要です。これらのメールは人間にとってわかりやすいもので、以前のメールのやりとりで得た顧客の感情に合わせてパーソナライズされます。

## タスク 2.1: 環境の設定

このタスクでは、環境を設定します。

In [ ]:
# Create a service client by name using the default session.
import json
import os
import boto3
import botocore

bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))

## タスク 2.2: テキストの生成

このタスクでは、Amazon Bedrock サービスにメールを生成させるための入力を準備します。

In [ ]:
# create the prompt
prompt_data = """
指示: AnyCompany のカスタマーサービスマネージャーである Bob から、当社のカスタマーサポートエンジニアが提供したサービスについて否定的なフィードバックを提供した顧客「John Doe」にメールを書いてください。"""

In [ ]:
body = json.dumps({
    "messages": [
        {
            "role": "user",
            "content": [{"text": prompt_data}]
        }
    ],
    "inferenceConfig": {
        "maxTokens": 8192,
        "stopSequences": [],
        "temperature": 0,
        "topP": 0.9
    }
}) 

次に、Amazon Nova Lite モデルを使用します。

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **注:** Amazon Nova Lite は、約 300k のトークンのコンテキストウィンドウをサポートし、次のパラメータを受け入れます。
- `messages`: ロールとコンテンツを持つメッセージオブジェクトの配列
- `inferenceConfig`: これらは、出力の生成時にモデルが考慮するパラメータです。

Amazon Bedrock API には、以下を受け入れる API `invoke_model` が用意されています。
- `modelId`: これは Amazon Bedrock で利用できるさまざまな基盤モデルのモデル ARN です
- `accept`: 入力リクエストのタイプ
- `contentType`: 出力のコンテンツタイプ
- `body`: プロンプトと設定から成る JSON 文字列

使用可能なテキスト生成モデル ID については、[ドキュメント](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids-arns.html)を参照してください。

## タスク 2.3: Amazon Nova Lite 言語モデルを呼び出す

このタスクでは、先ほど作成したプロンプトに基づいてモデルがどのように出力を生成するかを調べます。

### 出力生成の実行

このメールは、Amazon Nova Lite モデルが入力されたリクエストを理解し、内部に備わっているさまざまなモダリティに関する知識を活用することで生成されます。API へのリクエストは同期的に行われ、モデルによって出力全体が生成されるのを待ちます。

In [ ]:
#invoke model
modelId = 'amazon.nova-lite-v1:0' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'
outputText = "\n"
try:

    response = bedrock_client.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())

    outputText = response_body.get('output').get('message').get('content')[0].get('text')

except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubleshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error


In [ ]:
# Print the complete response from Nova Lite
print(outputText)


### 出力生成のストリーミング

出力がモデルによってチャンク形式で生成されるとき、Bedrock では出力のストリーミングもサポートします。この E メールは、ストリーミングオプションを使用してモデルを呼び出すことにより、生成されます。`invoke_model_with_response_stream` により `ResponseStream` が返され、それをユーザーが読めます。

In [ ]:
# invoke model with response stream
output = []
try:
    
    response = bedrock_client.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
    stream = response.get('body')
    
    i = 1
    if stream:
        for event in stream:
            chunk = event.get('chunk')
            if chunk:
                chunk_obj = json.loads(chunk.get('bytes').decode())
                if 'contentBlockDelta' in chunk_obj:
                    text = chunk_obj['contentBlockDelta']['delta']['text']
                    output.append(text)
                    print(f'\t\t\x1b[31m**Chunk {i}**\x1b[0m\n{text}\n')
                    i+=1
            
except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubleshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error

応答付きストリーミング方式は、モデルの出力をすばやく取得するのに役立ち、ユーザーが読んでいる間にサービスは出力を完了できます。これは、長いテキストを生成するようにモデルに要求するユースケースに役立ちます。生成されたすべてのチャンクを後で組み合わせて完全な出力を形成し、これをユースケースに使用できます。

In [ ]:
#combine output chunks
print('\t\t\x1b[31m**COMPLETE OUTPUT**\x1b[0m\n')
complete_output = ''.join(output)
print(complete_output)


これで、Amazon Bedrock API の基本的な公開を行う boto3 SDK を使用して実験ができました。この API を使用して、顧客からの否定的なフィードバックに応答するメールを生成するユースケースを見てきました。

### 自分でやってみよう
- プロンプトを特定のユースケースに合わせて変更し、さまざまなモデルの出力を評価します。
- トークンの長さを変えてみて、サービスのレイテンシーと応答性を理解します。
- さまざまなプロンプトエンジニアリングの原則を適用して、より良い出力を引き出します。

### クリーンアップ

このノートブックを完了しました。ラボの次の部分に進むために、以下を実行してください。

- このノートブックファイルを閉じて、**まとめ**に進みます。